In [116]:
from pathlib import Path
from transformers import BertTokenizer
from dataset import get_hypernyms_list_from_train, HypoDataset

data_path = Path('/home/hdd/data/hypernym/')
corpus_path = data_path / 'corpus.wikipedia-ru-2018-sample.token.txt'
hypo_index_path = data_path / 'index.full.wikipedia-ru-2018-sample.json'
train_set_path = data_path / 'train.cased.json'

model_path = Path('/home/hdd/models/rubert_v2/rubert_cased_L-12_H-768_A-12_v2/')
tokenizer_vocab_path = model_path / 'vocab.txt'

In [117]:
tokenizer = BertTokenizer(tokenizer_vocab_path, do_lower_case=False)
hype_list = get_hypernyms_list_from_train(train_set_path)

ds = HypoDataset(tokenizer,
                 corpus_path,
                 hypo_index_path,
                 train_set_path,
                 hype_list)

In [99]:
sample = ds.train_set[23]
sample

[['анкетирование'],
 [['сбор', 'собирание'],
  ['опрос', 'опрашивание'],
  ['задавание вопрос', 'вопрошание']],
 [['выражение', 'высказывание', 'изъявление'],
  ['задавание вопрос', 'вопрошание'],
  ['соединение'],
  ['обращение', 'обращенность'],
  ['сбор', 'собирание']]]

In [100]:
hypo = sample[0][0]
hypo

'анкетирование'

In [119]:
hypo in ds.hypo_index

True

In [118]:
hypo = 'эпилепсия'

In [120]:
index = ds.hypo_index[hypo]

In [131]:
count = 0
for corp_id, idx in index:
    print(count)
    count += 1
    text = ds.corpus[corp_id]
    print(text)
#     print(text.split()[idx])

0
Зыбочником также могли называть обманщика , хитреца и эпилепсию .

1
Доподлинно известно , что , находясь в самом центре этои борьбы , Иван Алексеевич ни разу не попытался принять в неи активное участие и не проявлял интереса к государственнои деятельности .

2
Прибыв в Лондон к Ленину , Троцкии стал постоянным сотрудником газеты , выступал с рефератами на собраниях эмигрантов и быстро приобрел известность .

3
Больные могут менять способ работы , изменять ход суждении , переключаться на другои вид деятельности .

4
Больные могут менять способ работы , изменять ход суждении , переключаться на другои вид деятельности .

5
Кроме того , резиденты производят эндопротезы , системы технического зрения , ранозаживляющие повязки , портативные метеорологические комплексы , системы управления воздушным движением , смеси и СВЧ - установки для ремонта асфальтобетонных покрытии , системы технологическои радиосвязи , новые марки полипропилена и полиэтилена , системы охранно - пожарнои и технологич

IndexError: list index out of range

In [104]:
from embedder import get_word_embeddings
from transformers import BertConfig, BertTokenizer, BertModel
import torch

In [123]:
config = BertConfig.from_pretrained(model_path / 'bert_config.json')
tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=False)
model = BertModel.from_pretrained(str(model_path / 'ptrubert.pt'), config=config)

emb_mat = model.embeddings.word_embeddings.weight

In [124]:
hype_embs = get_word_embeddings(hype_list, emb_mat, tokenizer=tokenizer)

In [137]:
for corp_id, idx in index[16:]:
    text = ds.corpus[corp_id]
    tokens = ['[CLS]']
    mask = [0.0]
    for n, token in enumerate(text.split()):
        current_tokens = tokenizer.tokenize(token)
        tokens.extend(current_tokens)
        mask_val = 1.0 if n == idx else 0.0
        mask.extend([mask_val] * len(current_tokens))
    tokens.append('[SEP]')
    mask.append(0.0)
    token_idxs = tokenizer.convert_tokens_to_ids(tokens)
    token_idxs_t = torch.tensor([token_idxs])
    h, v = model(token_idxs_t)
    mask_t = torch.tensor([mask])
    r = h * mask_t.unsqueeze(2)
    r = r.sum(1) / mask_t.sum(1)
    break
tokens

['[CLS]',
 'Американ',
 '##ски',
 '##и',
 'врач',
 '-',
 'ревмат',
 '##олог',
 'Роберт',
 'Кац',
 'счел',
 'нежел',
 '##ательным',
 'связывание',
 'любого',
 'упоминания',
 'припад',
 '##ков',
 'с',
 'эпилеп',
 '##си',
 '##еи',
 ',',
 'напоминая',
 'о',
 'существовании',
 'других',
 'причин',
 'припад',
 '##ков',
 '.',
 '[SEP]']

In [138]:
scores = r @ hype_embs.T

In [139]:
vals, indices = torch.topk(scores, 30)

In [140]:
for i in indices[0].tolist():
    print(i)
    print(hype_list[i])

33513
часть
1946
боль
25054
раздел
12425
медаль
2738
вид
10589
ком
14822
невролог
12686
мемориал
25374
район
9353
изоляция
26314
роль
32068
успех
18563
пар
33826
шлем
25306
разрыв
3065
внимание
21509
постель
9130
извилина
9446
инвалид
9714
инфоцентр
29381
стол
5146
глист
23315
проблема
6387
дефицит
33831
шов
4975
гематома
15400
нога
21111
помочь
26591
сайт
2746
вид рыба


In [111]:
hype_embs.shape

torch.Size([34480, 768])

In [112]:
len(hype_list)

34480

In [113]:
scores.shape

torch.Size([1, 34480])